# Cumulative luminosity profile from a *.fits* file

I used this function to calculate the cumulative luminosity profile for a model obtained as an optimization of the best-fit parameters using the program GALFIT.  
After reaching a good solution in the optimization phase, I choose to create the model with the parameters of the best-fit and, **not convolving with the PSF**, I started from this image to build the cumulative luminosity profile.  
I used this profile to evalue the effective radius $R_e$, i.e. the radius at which half of the total luminosity is emitted.

I defined a function which takes as input:
- the data of the *.fits* image:  
`hdu = fits.open('image.fits')[0]`  
    `img_data = hdu.data`
    
- the coordinates of the center $(x_0, y_0)$
- the lower extreme of the interval on which one wants to calculate the profile (it usually starts from the center, i.e  `min_interval=0`)
- the higher extreme of the interval on which one wants to calculate the profile. Be aware to not exceed the dimensions of the square image with the last radius. 
- the sampling step (in pixel) one wants to create the profile

In [ ]:
def cumulative_profile(data, center, min_interval, max_interval, step):
    number = int((max_interval- min_interval)/step +1) #numero di punti su cui valuto il profilo
    x, y = np.indices((data.shape[0], data.shape[1]))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2) #distanze dal centro
    r[ (r == 0. ) ] = 0.01  # metto un valore basso ma diverso da 0 al centro
    profile = np.zeros(len(np.linspace(min_interval, max_interval, number)))
    j=0
    for x in np.linspace(min_interval, max_interval, number):
        summ = np.sum(data[ (r <= x ) ]) #sommo i contributi solo dei pixel che sono dentro il raggio x
        profile[j]= summ    
        j = j+1
    return profile

### An easy example:

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#the parameters the user has to input
center = (100, 100)
min_interval, max_interval = 0. , 90.
step = 0.5

number = int((max_interval- min_interval)/step +1)
x_axis = np.linspace(min_interval, max_interval, number)

In [ ]:
#the image
hdu = fits.open('image.fits')[0]
img = hdu.data

#calculation of the cumulative light profile
cp = cumulative_profile(img1, center, min_interval, max_interval, step)

#plot del profilo
plt.figure(figsize=(10,5))
plt.plot(x_axis, cp)      #the profile 
plt.hlines(np.sum(img), min_interval, max_interval,  linestyles='dashed' , label = 'Total light')
plt.hlines(np.sum(img)*0.5 , min_interval, max_interval,  linestyles='solid' , label = 'Half light')
plt.grid()
plt.legend(loc='best')
plt.show()